<a href="https://colab.research.google.com/github/lucia-galiero/TICO-19_NMT_LLM/blob/main/groq_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 2.7 MB/s eta 0:00:00


In [ ]:
import os
from groq import Groq
import re

In [ ]:
from google.colab import userdata
userdata.get('GROQ_API_KEY')

'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'

In [ ]:
client= Groq(
    api_key= userdata.get('GROQ_API_KEY'),
)

#Read prompts from file and split into batches
def batch_prompts(file_path, batch_size):
    with open(file_path, 'r') as file:
        prompts = file.read().split('\n\n')  # Split by double newline

    # Batch prompts based on estimated token count (adjust as needed)
    current_batch = []
    current_batch_tokens = 0
    all_batches = []

    for prompt in prompts:
        prompt_tokens = len(prompt.split())  # Rough token estimation
        if current_batch_tokens + prompt_tokens <= batch_size:
            current_batch.append(prompt)
            current_batch_tokens += prompt_tokens
        else:
            all_batches.append(current_batch)
            current_batch = [prompt]
            current_batch_tokens = prompt_tokens

    if current_batch:  # Add any remaining prompts
        all_batches.append(current_batch)

    return all_batches


file_path = '/content/LLAMA_2S_prompts.txt'
batch_size = 1000

batches = batch_prompts(file_path, batch_size)

for batch in batches:
    # Combine prompts into a single string for the API request
    combined_prompt = "\n\n".join(batch) + "\nItalian:"

    # Make the API request
    completion = client.chat.completions.create(
        model="llama-3.2-90b-vision-preview",
        messages=[
            {
                "role": "system",
                "content": "You need to translate sentences from English to Italian from scientific articles on Covid-19, for each we’ll provide some examples of the style and terminology used, and a translation provided by a machine translation system (MT), which might contain errors. Please output your best translation only, with no other explanation or comment\n\n "
            },
            {
                "role": "user",
                "content": combined_prompt
            }
        ],
        temperature=1,
        max_completion_tokens=1024,
        top_p=1,
        stream=False,
        stop=None,
    )

    # Print or store the results
    print(completion.choices[0].message.content)
    print("-" * 20)  # Separator between batch results


Stime Preliminari della Prevalenza di Alcune Condizioni di Salute Sottostanti Selezionate tra i Pazienti con Malattia del Coronavirus 2019 — Stati Uniti, February 12–March 28, 2020.

Rapporti provenienti dalla Cina e dall'Italia suggeriscono che i fattori di rischio per la malattia grave includono l'età avanzata e la presenza di almeno una o più delle seguenti condizioni di salute sottostanti.

Tra questi 7.162 casi, 2.692 (37,6%) pazienti avevano una o più condizioni di salute sottostanti o fattori di rischio, e 4.470 (62,4%) non avevano nessuna di queste condizioni riportate.

Questi risultati preliminari suggeriscono che negli Stati Uniti, le persone con condizioni di salute sottostanti o altri fattori di rischio riconosciuti per esiti gravi a seguito di infezioni respiratorie sembrano essere ad un maggior rischio per una malattia grave da COVID-19 rispetto alle persone senza queste condizioni.
--------------------
Per i casi con date di insorgenza mancanti, la data di insorgenza è 